This is the setup cell

In [1]:
import datetime
import pathlib
import time
import math

import obspy
from obspy.signal import PPSD
import scipy
import matplotlib.pyplot as plt

#Run this line if you want interactive plots
%matplotlib qt

import numpy as np

import sprit

In [2]:
dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake\Test12_AM.RAC84.00.2023-02-15_2132-2200.mseed'
trimDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\ChampaignCo\Mahomet2023\Data\RawData"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\RAC84"
dPath = r"C:\Users\balikian\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\UOFI1_AM.RAC84.00.2023-02-15_1704-1734.mseed"

params = sprit.input_param( acq_date='2023-02-15',
                            #starttime = obspy.UTCDateTime('2023-02-06T21:32:00.00'),
                            #endtime = obspy.UTCDateTime('2023-02-06T22:00:00.00'),
                            tzone = 'UTC',#or 'local', #or 'UTC'
                            dst=False,
                            lon = -88.2290526,
                            lat =  40.1012122,
                            elevation = 755,
                            site = 'HVSR Site',
                            dataPath = dPath
                            )

No metadata file specified!
Using default metadata file for Raspberry Shake v.7 contained in repository at
 c:/Users/balikian/LocalData/CodesScripts/Github/Sprite-HV/resources/raspshake_metadata.inv


In [3]:
params = sprit.get_metadata(params)
stream = sprit.fetch_data(params=params)

Day of Year: 46


c:\ProgramData\Anaconda3\envs\seismic38\lib\site-packages\obspy\core\inventory\network.py:249: UserWarning: Found more than one matching response. Returning first.
  warnings.warn(msg)


In [4]:
xwindows = sprit.select_windows(stream)

In [8]:
def select_windows(input):
    """_summary_

    Parameters
    ----------
    hvsr_dict : dict
        Dictionary containing all the hvsr information.

    Returns
    -------
    xWindows : list
        List of two-item lists containing start and end times of windows to be removed.
    """
    from matplotlib.backend_bases import MouseButton
    import matplotlib.pyplot as plt
    import matplotlib

    if type(input) is dict:
        fig, ax = sprit.hvplot(hvsr_dict=input, kind='spec', returnfig=True, cmap='turbo')
        stream = []
    elif isinstance(input, obspy.core.stream.Stream):
        fig, ax = __plot_specgram_stream(input, component=['Z'])
    elif isinstance(input, obspy.core.trace.Trace):
        fig, ax = __plot_specgram_stream(input)
        plt.specgram()

    global clickNo
    global x0
    x0=0
    clickNo = 0
    xWindows = []
    pathList = []
    windowDrawn = []
    winArtist = []
    lineArtist = []

    global on_click
    def on_click(event):
        global clickNo
        global x0
        if event.button is MouseButton.RIGHT:
            global remove_on_right
            def remove_on_right(event, xWindows, pathList, windowDrawn, winArtist,  lineArtist, fig=fig, ax=ax):
                if xWindows is not None:
                    for i, xWins in enumerate(xWindows):
                        #for j, k in enumerate(ax.keys()):
                        if event.xdata > xWins[0] and event.xdata < xWins[1]:
                            linArtists = lineArtist[i]
                            winArtists = winArtist[i]
                            pathList.pop(i)
                            for j, a in enumerate(linArtists):
                                w = winArtists[j]
                                winArtist[i][j].remove()#.pop(i)
                                lineArtist[i][j][0].remove()#.pop(i)#[i].pop(j)
                                lineArtist[i][j][1].remove()
                            windowDrawn.pop(i)
                            lineArtist.pop(i)#[i].pop(j)
                            winArtist.pop(i)#[i].pop(j)
                        xWindows.pop(i)
                fig.canvas.draw() 
                #draw_windows(event, pathlist=pathList)
            remove_on_right(event, xWindows, pathList, windowDrawn, winArtist, lineArtist)

        if event.button is MouseButton.LEFT:            
            def draw_boxes(event, clickNo, xWindows, pathList, windowDrawn, winArtist, lineArtist, fig=fig, ax=ax, x0=x0):
                #Create an axis dictionary if it does not already exist so all functions are the same
                if len(ax) > 1:
                    multAxes = True
                    if type(ax) is not dict:
                        axDict = {}
                        for i, a in enumerate(ax):
                            axDict[str(i)] = a
                        ax = axDict
                else:
                    multAxes = False
                    ax = {'a':ax}
                
                #if event.inaxes!=ax: return
                #y0, y1 = ax.get_ylim()
                y0 = []
                y1 = []
                kList = []
                for k in ax.keys():
                    kList.append(k)
                    y0.append(ax[k].get_ylim()[0])
                    y1.append(ax[k].get_ylim()[1])
                #else:
                #    y0 = [ax.get_ylim()[0]]
                #    y1 = [ax.get_ylim()[1]]

                if clickNo == 0:
                    #y = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 2)
                    x0 = event.xdata
                    clickNo = 1   
                    lineArtist.append([])
                    winNums = len(xWindows)
                    for i, k in enumerate(ax.keys()):
                        linArt = ax[k].axvline(x0, 0, 1, color='k', linewidth=1, zorder=100)
                        lineArtist[winNums].append([linArt, linArt])
                    #else:
                    #    linArt = plt.axvline(x0, y0[i], y1[i], color='k', linewidth=1, zorder=100)
                    #    lineArtist.append([linArt, linArt])
                else:
                    x1 = event.xdata
                    clickNo = 0

                    windowDrawn.append([])
                    winArtist.append([])  
                    pathList.append([])
                    winNums = len(xWindows)
                    for i, key in enumerate(kList):
                        path_data = [
                            (matplotlib.path.Path.MOVETO, (x0, y0[i])),
                            (matplotlib.path.Path.LINETO, (x1, y0[i])),
                            (matplotlib.path.Path.LINETO, (x1, y1[i])),
                            (matplotlib.path.Path.LINETO, (x0, y1[i])),
                            (matplotlib.path.Path.LINETO, (x0, y0[i])),
                            (matplotlib.path.Path.CLOSEPOLY, (x0, y0[i])),
                        ]
                        codes, verts = zip(*path_data)
                        path = matplotlib.path.Path(verts, codes)

                        windowDrawn[winNums].append(False)
                        winArtist[winNums].append(None)

                        pathList[winNums].append(path)
                        draw_windows(event=event, pathlist=pathList, ax_key=key, windowDrawn=windowDrawn, winArtist=winArtist)
                        linArt = plt.axvline(x1, 0, 1, color='k', linewidth=0.5, zorder=100)

                        [lineArtist[winNums][i].pop(-1)]
                        lineArtist[winNums][i].append(linArt)
                    x_win = [x0, x1]
                    x_win.sort() #Make sure they are in the right order
                    xWindows.append(x_win)
                fig.canvas.draw()
                return clickNo, x0
    
            global draw_windows
            def draw_windows(event, pathlist, ax_key, windowDrawn, winArtist, fig=fig, ax=ax):
                winNums = len(xWindows)
                for i, pa in enumerate(pathlist):
                    for j, p in enumerate(pa): 
                        if windowDrawn[i][j]:
                            pass
                        else:
                            patch = matplotlib.patches.PathPatch(p, facecolor='k', alpha=0.75)                            
                            winArt = ax[ax_key].add_patch(patch)
                            windowDrawn[i][j] = True
                            winArtist[i][j] = winArt

                if event.button is MouseButton.RIGHT:
                    fig.canvas.draw() 
            
            clickNo, x0 = draw_boxes(event, clickNo, xWindows, pathList, windowDrawn, winArtist, lineArtist, x0=x0)    

    fig.canvas.mpl_connect('button_press_event', on_click)
    plt.show()
    return xWindows

response = select_windows(input=stream)

In [7]:
def __plot_specgram_stream(stream, params=None, component='Z', stack_type='linear', detrend='mean', dbscale=True, return_fig=True, cmap_per=[0.1,0.9], **kwargs):
    import matplotlib.dates as mdates
    import matplotlib
    og_stream = stream.copy()

    traceList = []
    maxStartTime = obspy.UTCDateTime(-1e10) #Go back pretty far (almost 400 years) to start with
    minEndTime = obspy.UTCDateTime(1e10)
    for comp in ['E', 'N', 'Z']:
        tr = stream.select(component=comp).copy()
        if comp in component:
            traceList.append(tr[0])
        if tr[0].stats.starttime > maxStartTime:
            maxStartTime = tr[0].stats.starttime
        if tr[0].stats.endtime < minEndTime:
            minEndTime = tr[0].stats.endtime

    for tr in traceList:
        tr.trim(starttime=maxStartTime, endtime=minEndTime)
    og_stream.trim(starttime=maxStartTime, endtime=minEndTime)      

    stream = obspy.Stream(traceList)
    stream.stack(group_by='all', npts_tol=200, stack_type=stack_type)  

    mosaic = [['spec'],['spec'],['spec'],
              ['spec'],['spec'],['spec'],
              ['signalz'],['signalz'], ['signaln'], ['signale']]
    fig, ax = plt.subplot_mosaic(mosaic, sharex=True)  
    
    #fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True)  

    data = stream[0].data
    sample_rate = stream[0].stats.sampling_rate
    if 'title' in kwargs.keys():
        title=kwargs['title']
    else:
        title=stream[0].id

    if 'cmap' in kwargs.keys():
        cmap=kwargs['cmap']
    else:
        cmap='turbo'

    if params is None:
        hvsr_band = [0.4, 40]
    else:
        hvsr_band = params['hvsr_band']
    ymin = hvsr_band[0]
    ymax = hvsr_band[1]

    spec, freqs, times, im = ax['spec'].specgram(x=data, Fs=sample_rate, detrend=detrend, scale_by_freq=True, scale='dB')
    im.remove()

    difference_array = freqs-ymin
    for i, d in enumerate(difference_array):
        if d > 0:
            minfreq = difference_array[i-1]
            if i-1 < 0:
                i=1
            minfreqInd = i-1
            break
            
    difference_array = freqs-ymax
    for i, d in enumerate(difference_array):
        if d > 0:
            maxfreqInd = i-1
            maxfreq = difference_array[i-1]
            break

    array_displayed = spec[minfreqInd:maxfreqInd,:]
    freqs_displayed = freqs[minfreqInd:maxfreqInd]
    #im.set_data(array_displayed)
    vmin = np.percentile(array_displayed, cmap_per[0]*100)
    vmax = np.percentile(array_displayed, cmap_per[1]*100)
    
    sTime = stream[0].stats.starttime
    timeList = {}
    mplTimes = {}
    og_stream.decimate(10)
    for i, tr in enumerate(og_stream):
        key = tr.stats.component
        timeList[key] = []
        mplTimes[key] = []
        for t in tr.times():
            t = sTime + t
            timeList[key].append(t)
            mplTimes[key].append(t.matplotlib_date)
    
    for i, k in enumerate(mplTimes.keys()):
        if i == 0:
            xmin = np.min(mplTimes[k])
            xmax = np.max(mplTimes[k])
        else:
            if xmin > np.min(mplTimes[k]):
                xmin = np.min(mplTimes[k])
            if xmax < np.max(mplTimes[k]):
                xmax = np.max(mplTimes[k])         
                   
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    im = ax['spec'].imshow(array_displayed, norm=norm, cmap=cmap, aspect='auto', interpolation=None, extent=[xmin,xmax,ymax,ymin])
    plt.gca()
    ax['spec'].set_xlim([xmin, xmax])
    ax['spec'].set_ylim([ymin, ymax])
    ax['spec'].semilogy() 
    
    #cbar = plt.colorbar(mappable=im)
    #cbar.set_label('Power Spectral Density [dB]')
    #stream.spectrogram(samp_rate=sample_rate, axes=ax, per_lap=0.75, log=True, title=title, cmap='turbo', dbscale=dbscale, show=False)
    
    ax['spec'].xaxis_date()
    ax['signalz'].xaxis_date()
    ax['signaln'].xaxis_date()
    ax['signale'].xaxis_date()
    #tTicks = mdates.MinuteLocator(interval=5)
    #ax[0].xaxis.set_major_locator(tTicks)
    ax['signale'].xaxis.set_major_locator(mdates.MinuteLocator(interval=5))
    ax['signale'].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax['signale'].xaxis.set_minor_locator(mdates.MinuteLocator(interval=1))
    plt.tick_params(axis='x', labelsize=8)
    
    ax['signalz'].plot(mplTimes['Z'],og_stream.select(component='Z')[0].data, color='k', linewidth=0.25)
    ax['signaln'].plot(mplTimes['N'],og_stream.select(component='N')[0].data, color='k', linewidth=0.1)
    ax['signale'].plot(mplTimes['E'],og_stream.select(component='E')[0].data, color='k', linewidth=0.1)
    plt.gca()
    
    for comp in mplTimes.keys():
        stD = np.std(og_stream.select(component=comp)[0].data)
        dmed = np.nanmedian(og_stream.select(component=comp)[0].data)
        key = 'signal'+comp.lower()
        ax[key].set_ylim([dmed-2.5*stD, dmed+2.5*stD])
    
    if params is None:
        plt.suptitle('HVSR Site: Spectrogram')
    else:
        plt.suptitle(params['site']+': Spectrogram')
    
    day = "{}-{}-{}".format(stream[0].stats.starttime.year, stream[0].stats.starttime.month, stream[0].stats.starttime.day)
    plt.xlabel('UTC Time \n'+day)

    plt.rcParams['figure.dpi'] = 100
    plt.rcParams['figure.figsize'] = (5,4)
    
    #axSpan = fig.add_subplot([0,0,1,0.4])
    #axSpan.set_axis_off()
    #axSpan.set_ylabel('Original signal') 
    
    #fig.tight_layout()
    plt.show()
    if return_fig:
        return fig, ax
    return

#stream = sprit.fetch_data(params=params)
#stream = sprit.trim_data(stream, params, export_dir=r"C:\Users\riley\OneDrive\Documents")

#fig, ax = __plot_specgram_stream(stream, components="Z", detrend='linear', return_fig=True)

In [4]:
params = sprit.generate_ppsds(params=params, stream=stream, ppsd_length=60, overlap=0.5, period_step_octaves=0.01)
hvsr_dict = sprit.process_hvsr(params=params, method=4, resample=False, smooth=10)
hvsr_dict = sprit.check_peaks(hvsr_dict=hvsr_dict)

In [5]:
plt.ion()
sprit.hvplot(hvsr_dict, kind='HVSR', show=True)

In [16]:
from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

#fig, ax = plt.subplots()


fig, ax = sprit.hvplot(hvsr_dict=hvsr_dict, kind='spec', returnfig=True, cmap='turbo')

clickNo = 0    
xWindows = []
pathList = []
windowDrawn = []
winArtist = []
lineArtist = []

def on_click(event, fig=fig, ax=ax):
    if event.button is MouseButton.RIGHT:
        remove_on_right(event)
    if event.button is MouseButton.LEFT:
        draw_boxes(event)

def draw_boxes(event, fig=fig, ax=ax):
    global clickNo
    global x0
    global xWindows
    global pathList
    global windowDrawn
    global winArtist
    global lineArtist

    if event.inaxes!=ax: return
    #y0, y1 = ax.get_ylim()
    y0=0
    y1=50
    
    if clickNo == 0:
        #y = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 2)
        x0 = event.xdata
        clickNo += 1
        linArt = plt.axvline(x0, y0, y1, linewidth=0.5, color='k', zorder=100)
        lineArtist.append([linArt, linArt])
    else:
        x1 = event.xdata
        clickNo = 0

        path_data = [
            (matplotlib.path.Path.MOVETO, (x0, y0)),
            (matplotlib.path.Path.LINETO, (x1, y0)),
            (matplotlib.path.Path.LINETO, (x1, y1)),
            (matplotlib.path.Path.LINETO, (x0, y1)),
            (matplotlib.path.Path.LINETO, (x0, y0)),
            (matplotlib.path.Path.CLOSEPOLY, (x0, y0)),
        ]

        codes, verts = zip(*path_data)
        path = matplotlib.path.Path(verts, codes)

        x_win = [x0, x1]
        xWindows.append(x_win)
        pathList.append(path)
        windowDrawn.append(False)
        winArtist.append(None)
        [lineArtist[-1].pop()]
        draw_windows(event=event, pathlist=pathList)
        linArt = plt.axvline(x1, y0, y1, color='k',linewidth=0.5, zorder=100)
        lineArtist[-1].append(linArt)
    fig.canvas.draw() 

def draw_windows(event, pathlist, fig=fig, ax=ax):
    for i, p in enumerate(pathList):
        if windowDrawn[i]:
            pass
        else:
            patch = matplotlib.patches.PathPatch(p, facecolor='k', alpha=0.75)
            winArt = ax.add_patch(patch)
            windowDrawn[i] = True
            winArtist[i] = winArt
    if event.button is MouseButton.RIGHT:
        fig.canvas.draw() 

def remove_on_right(event, fig=fig, ax=ax, xWindows=xWindows):
    if xWindows is not None:
        for i, xWins in enumerate(xWindows):
            if event.xdata > xWins[0] and event.xdata < xWins[1]:
                xWindows.pop(i)
                pathList.pop(i)
                winArtist[i].remove()
                winArtist.pop(i)
                windowDrawn.pop(i)
                lineArtist[i][0].remove()
                lineArtist[i][1].remove()
                lineArtist.pop(i)
    fig.canvas.draw() 
    #draw_windows(event, pathlist=pathList)

fig.canvas.mpl_connect('button_press_event', on_click)
plt.show()